<a href="https://colab.research.google.com/github/tikhomirov11/min-max/blob/main/%D0%BC%D0%B8%D0%BD_%D0%BC%D0%B0%D0%BA%D1%81_%D0%B8%D1%82%D0%BE%D0%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime
import numpy as np
from scipy.stats import norm
import statistics


In [ ]:
#данные, которые необходимо ввести пользователю

#название склада, для которого рассчитываем мин-максы
name='МСК'

#Время на разгрузку товара - влияет на [Время на разгрузку товара], дни
razgruzka = 3

#Период доставки от инфора до склада - влияет на [Время на разгрузку товара],дни
dostavka = 3


#Минимальное количество заказов	 - влияет на [мин 2] и [макс 3]
min_kolvo_zakazov=2

#Количество месяцев с заказами - влияет на [макс 3]
kolvo_mes_s_zakazami=2

#Количество месяцев ТЗ - влияет на [макс 1]
kolvo_mesyacev_tz=4

#Отсечка всплесков - влияет на [Всего шт в заказах за анализируемый период с учетом отсечения всплесков]
otsechka_vspleskov=2
#Обеспечение спроса - влияет на [макс 1] и [макс 2]
obespec_sprosa=0.99

#Максимальный запас, мес- влияет на [макс 3]
max_zapas=6

#Минимальный остаток не больше максимального остатка - влияет на [мин 4]
min_menshe_maxa_ne_bolee_chem_na=0.25

#количество месяцев в анализируемом периоде
an_period=6




In [ ]:
#прописываем полный путь, где лежат файлы. Пример - https://docs.google.com/spreadsheets/d/16lIcR32q-Zyiv45wrZj3x-6pWdczVIRy8XGNEGg_hCI/edit#gid=204408087

# Важно! - это 3 отдельных файла.

#в примере лист - Заявки
table_zayavki=pd.read_excel('Москва скрипт итог.xlsx')

#в примере лист - Остатки инфор
table_infor_ostatki=pd.read_excel('Остатки товаров.xlsx')

#в примере лист - Средний остаток
table_sred_ostatki=pd.read_excel('Средний остаток.xlsx')


In [ ]:
#вычисляем обратную стандартную функцию нормального распределения
normstobr=norm.ppf(obespec_sprosa)


koren=kolvo_mesyacev_tz**(0.5)

#Общее время на доставку - влияет на [мин 1]

obwee_vremya_na_dostavku=razgruzka+dostavka


#вычисляем текущую дату
dt_now = datetime.datetime.now()

#количество месяцев, которые влияют больше всего
mes_vl=int(an_period/2)

In [ ]:
#заменяем пустые ячейки на 0

table_zayavki = table_zayavki.fillna(0)


#добавляем пустой столбец "Месяцев с создания"
table_zayavki['Месяцев с создания']=''

#заполняем столбец "Месяцев с создания"
#формула расчета: (сегодня - дата создания)/30, то есть считаем количество прошедших дней и делим на 30, чтобы получить месяцы
#функция datetime.datetime.strptime - переводит дату в нужный формат

for i in range(len(table_zayavki)):
    table_zayavki['Месяцев с создания'][i]=(dt_now-datetime.datetime.strptime(table_zayavki['Дата создания'][i], '%d.%m.%Y %H:%M:%S')).days/30


#добавляем пустой столбец "Среднее количество шт в заказах за анализируемый период с учетом даты заведения"

table_zayavki['Среднее количество шт в заказах за анализируемый период с учетом даты заведения']=''

#заполняем столбец "Среднее количество шт в заказах за анализируемый период с учетом даты заведения"

for i in range(len(table_zayavki)):
    #если  [месяцев с создания] меньше периода анализа, то делим количество шт в заказах на количество месяцев с создания
    if table_zayavki['Месяцев с создания'][i]<an_period:
        table_zayavki['Среднее количество шт в заказах за анализируемый период с учетом даты заведения'][i]=table_zayavki['Итого'][i]/table_zayavki['Месяцев с создания'][i]
    #иначе делим количество шт в заказах на количество месяцев в анализируемом периоде
    else:
        table_zayavki['Среднее количество шт в заказах за анализируемый период с учетом даты заведения'][i]=table_zayavki['Итого'][i]/an_period



#добавляем пустой столбец "Количество месяцев с заказами с учетом даты заведения"

table_zayavki['Количество месяцев с заказами с учетом даты заведения']=''


#считаем количество месяцев, у которых значение больше 0
for j in range(len(table_zayavki)):
    t=0

    if table_zayavki['Месяцев с создания'][j]>an_period:
        for i in range(1,an_period+1):
            if table_zayavki[i][j]>0:
                t+=1


    else:
        for u in range(an_period+1-int(table_zayavki['Месяцев с создания'][j]),an_period+1):
            if table_zayavki[u][j]>0:
                t+=1

    #заполняем столбец "Количество месяцев с заказами с учетом даты заведения"
    table_zayavki['Количество месяцев с заказами с учетом даты заведения'][j]=t


#добавляем пустой столбец "Медиана заказов с учетом даты заведения"

table_zayavki['Медиана заказов с учетом даты заведения']=''

#добавляем пустой столбец "80% всех значений в месяцах"
table_zayavki['80% всех значений в месяцах']=''


#добавляем пустой столбец "Медиана за последнюю четверть периода"

table_zayavki['Медиана за последнюю четверть периода']=''


#вычисляем значение, которое является 80% всех значений

for i in range(len(table_zayavki)):
    a=[]
    #если месяцев с создания меньше анализируемого периода, то вычисляем только по периоду с даты создания номенклатуры
    if table_zayavki['Месяцев с создания'][i]<an_period:
        for j in range(an_period-int(table_zayavki['Месяцев с создания'][i]),an_period+1):
            a.append(table_zayavki[j][i])
        table_zayavki['Медиана заказов с учетом даты заведения'][i]=sum(a)/len(a)
        sorted_list = sorted(a)
        if len(a)>=4:
            table_zayavki['80% всех значений в месяцах'][i]=sorted_list[-4]
        else:
            table_zayavki['80% всех значений в месяцах'][i]=sorted_list[-len(a)]

    # иначе вычисляем по всему периоду
    else:
        for j in range(1,an_period+1):
            a.append(table_zayavki[j][i])
        table_zayavki['Медиана заказов с учетом даты заведения'][i]=sum(a)/len(a)
        sorted_list = sorted(a)
        if len(a)>=4:
            table_zayavki['80% всех значений в месяцах'][i]=sorted_list[-4]
        else:
            table_zayavki['80% всех значений в месяцах'][i]=sorted_list[-len(a)]



#добавляем пустой столбец "Сумма продаж за влияющий периодгод с учетом всплесков"

table_zayavki['Сумма продаж за влияющий период с учетом всплесков']=''

#вычисляем количество заказов за [влияющий период]
for i in range(len(table_zayavki)):
    a=[]
    for j in range(an_period-mes_vl,an_period+1):
        a.append(table_zayavki[j][i])
    table_zayavki['Сумма продаж за влияющий период с учетом всплесков'][i]=sum(a)


#вычисляем медиану за последнюю четверть периода, с учетом даты заведения позиции

for i in range(len(table_zayavki)):
    a=[]
    if table_zayavki['Месяцев с создания'][i]<(an_period/4):
        for j in range(an_period+1-int(table_zayavki['Месяцев с создания'][i]),an_period+1):
            a.append(table_zayavki[j][i])
        table_zayavki['Медиана за последнюю четверть периода'][i]=np.median(a)

    else:
        for j in range(int(an_period-(an_period/4)),an_period+1):
            a.append(table_zayavki[j][i])
        table_zayavki['Медиана за последнюю четверть периода'][i]=np.median(a)



#добавляем пустой столбец "Сумма значений с учетом всплесков"

table_zayavki['Сумма значений с учетом всплесков']=''



#сглаживаем данные
#если значение в месяце больше 80% всех значений в месяцах, то заменяем его на 80%-е значение
#далее суммируем полученные значения

for i in range(len(table_zayavki)):
    sum_vspl=0
    if table_zayavki['Месяцев с создания'][i]<an_period:
        for j in range(an_period-int(table_zayavki['Месяцев с создания'][i]),an_period+1):
            if table_zayavki['80% всех значений в месяцах'][i]>table_zayavki[j][i]:
                sum_vspl+=table_zayavki[j][i]
            else:
                sum_vspl+=table_zayavki['80% всех значений в месяцах'][i]


        table_zayavki['Сумма значений с учетом всплесков'][i]=sum_vspl


    else:
        for j in range(1,an_period+1):
            if table_zayavki['80% всех значений в месяцах'][i]>table_zayavki[j][i]:
                sum_vspl+=table_zayavki[j][i]
            else:
                sum_vspl+=table_zayavki['80% всех значений в месяцах'][i]


        table_zayavki['Сумма значений с учетом всплесков'][i]=sum_vspl



#добавляем пустой столбец "Всего шт в заказах за анализируемый период с учетом отсечения всплесков"

table_zayavki['Всего шт в заказах за анализируемый период с учетом отсечения всплесков']=''


#создаем столбцы 1_сглаживание, 2_сглаживание для дальнейшего заполнения сглаженными значениями
for i in range(1,an_period+1):
    hd=str(i)+'_сглаживание'
    table_zayavki[hd]=''






#сглаживаем данные
#если значение больше медианы в [отсечка всплесков], то заменяем на медианное значение

for i in range(len(table_zayavki)):

    summ_p=0
    for j in range(1,an_period+1):
        name_st=str(j)+'_сглаживание'
        if (table_zayavki[j][i]/table_zayavki['Медиана заказов с учетом даты заведения'][i])>otsechka_vspleskov:
            p=table_zayavki['Медиана заказов с учетом даты заведения'][i]
            #полученное значение округляем в большую сторону до целого
            table_zayavki[name_st][i]=int(p) + (1 if p%1 > 0 else 0)
            summ_p+=int(p) + (1 if p%1 > 0 else 0)

        else:
            p=table_zayavki[j][i]
            table_zayavki[name_st][i]=int(p) + (1 if p%1 > 0 else 0)
            summ_p+=int(p) + (1 if p%1 > 0 else 0)

    #записываем сумму полученных значений за анализируемый период
    table_zayavki['Всего шт в заказах за анализируемый период с учетом отсечения всплесков'][i]=summ_p



#переименовывем столбец Итого.1
table_zayavki['Количество заказов за анализируемый период']=table_zayavki['Итого.1']


#вычисляем средний остаток инфора за последние полгода
table_sred_ostatki['Средний остаток на Инфор за последние полгода']=(table_sred_ostatki[1]+table_sred_ostatki[2]+table_sred_ostatki[3]+table_sred_ostatki[4]+table_sred_ostatki[5]+table_sred_ostatki[6])/6



#соединяем 2 таблицы: таблицу с заявками и таблицу со средними остатками
df=table_zayavki.merge(table_sred_ostatki,left_on='Артикул',right_on='Артикул',how='outer')


#пустые поля заменяем на 0
df = df.fillna(0)


#оставляем в таблице только поля, у которых наименование не равно 0, могли появиться после объединения таблиц
table_zayavki=df[df['Наименование']!=0]

#соединяем 2 таблицы: таблицу с заявками и таблицу с остатками инфора

df=table_zayavki.merge(table_infor_ostatki,left_on='Артикул',right_on='Артикул',how='outer')

#пустые поля заменяем на 0
df = df.fillna(0)

#оставляем в таблице только поля, у которых наименование не равно 0, могли появиться после объединения таблиц
table_zayavki=df[df['Наименование']!=0]

#переименовывем столбец Количество
table_zayavki['Остаток на Инфоре на сегодня']=table_zayavki['Количество']

#формула для прогноза предполагает работу с 5 месяцами, поэтому нам нужно выявить максимальное количество 5месячных блоков
kolvo_mes_dlya_prognoza=(an_period//5)*5

#создаем столбцы для заполнения прогнозными данными

for i in range(1,kolvo_mes_dlya_prognoza+1):
    hd=str(i)+'_сглаживание_прогноз'
    table_zayavki[hd]=''


#вычисляем количество месяцев, которые пропускаем (если есть неполный 5месячный блок)
raznica=an_period-kolvo_mes_dlya_prognoza




#сглаживаем по 5 точкам

#подробнее про метод - http://eokd.tolgas.ru/euk/iz/prac1.html

for j in range(len(table_zayavki)):
    for i in range(1,kolvo_mes_dlya_prognoza,5):

        a_1=table_zayavki[str(i+raznica)+'_сглаживание'][j]
        a_2=table_zayavki[str(i+raznica+1)+'_сглаживание'][j]
        a_3=table_zayavki[str(i+raznica+2)+'_сглаживание'][j]
        a_4=table_zayavki[str(i+raznica+3)+'_сглаживание'][j]
        a_5=table_zayavki[str(i+raznica+4)+'_сглаживание'][j]


        table_zayavki[str(i)+'_сглаживание_прогноз'][j]=(3*a_1+2*a_3+a_4-a_5)/5
        table_zayavki[str(i+1)+'_сглаживание_прогноз'][j]=(4*a_1+3*a_2+2*a_3+a_4)/10
        table_zayavki[str(i+2)+'_сглаживание_прогноз'][j]=(a_1+a_2+a_3+a_4+a_5)/5
        table_zayavki[str(i+3)+'_сглаживание_прогноз'][j]=(a_2+2*a_3+3*a_4+4*a_5)/10
        table_zayavki[str(i+4)+'_сглаживание_прогноз'][j]=(-a_1+a_3+2*a_4+3*a_5)/5




    #добавляем пустой столбец "Среднее по сглаженному"

table_zayavki['Среднее по сглаженному']=''


#считаем среднее по сглаженным точкам по методу 5 точек, с учетом даты создания

for j in range(len(table_zayavki)):
    if table_zayavki['Медиана заказов с учетом даты заведения'][j]<1:
        fin=table_zayavki['Медиана за последнюю четверть периода'][j]
    else:
        if table_zayavki['Количество месяцев с заказами с учетом даты заведения'][j]>kolvo_mes_s_zakazami:
            fin=table_zayavki['Сумма значений с учетом всплесков'][j]/table_zayavki['Количество месяцев с заказами с учетом даты заведения'][j]
        else:
            if table_zayavki['Месяцев с создания'][j]>kolvo_mes_dlya_prognoza:
                sum_sgla=0
                for i in range(1,kolvo_mes_dlya_prognoza+1):
                    sum_sgla+=table_zayavki[str(i)+'_сглаживание_прогноз'][j]
                fin=sum_sgla/kolvo_mes_dlya_prognoza
            else:
                if table_zayavki['Месяцев с создания'][j]<5:
                    fin=table_zayavki['Среднее количество шт в заказах за анализируемый период с учетом даты заведения'][j]
                else:
                    if table_zayavki['Месяцев с создания'][j]<kolvo_mes_dlya_prognoza:
                        sum_sgla=0
                        t=0
                        for i in range(int(1+kolvo_mes_dlya_prognoza-(table_zayavki['Месяцев с создания'][j]//5)*5),kolvo_mes_dlya_prognoza+1):
                            sum_sgla+=table_zayavki[str(i)+'_сглаживание_прогноз'][j]
                            t+=1
                        fin=sum_sgla/t


    table_zayavki['Среднее по сглаженному'][j]=fin



#добавляем пустые столбцы для каждого этапа расчета мин-максов
table_zayavki['мин1']=''
table_zayavki['мин2']=''
table_zayavki['мин3']=''
table_zayavki['мин4']=''
table_zayavki['макс1']=''
table_zayavki['макс2']=''
table_zayavki['макс3']=''


for j in range(len(table_zayavki)):
    #макс1

    #собираем итоговые сглаженные значения в один список
    apo=[]
    if table_zayavki['Месяцев с создания'][j]>kolvo_mes_dlya_prognoza:
        for i in range(1,kolvo_mes_dlya_prognoza):
            apo.append(table_zayavki[str(i)+'_сглаживание_прогноз'][j])

    else:
        for i in range(int(1+kolvo_mes_dlya_prognoza-(table_zayavki['Месяцев с создания'][j]//5)*5),kolvo_mes_dlya_prognoza+1):
                apo.append(table_zayavki[str(i)+'_сглаживание_прогноз'][j])


    #вычисляем стандартное отклонение по сглаженным значениям
    #Стандартное отклонение — это мера того, насколько широко разбросаны точки данных относительно их среднего.
    if apo==[]:
        std_otkl=0
    else:
        std_otkl = statistics.stdev(apo)

    #вычисляем первый этап макс, учитывая среднемесячный спрос, вероятность отклонения и количество месяцев запаса товара
    max1=table_zayavki['Среднее по сглаженному'][j]+koren*std_otkl*normstobr

    #полученное значение округляем до целого значения
    table_zayavki['макс1'][j]=int(max1) + (1 if max1%1 >= 0.5 else 0) #тут меняю int на float
    #max2

    #если товар импортный и полученное значение в макс 1 меньше среднего остатка на инфоре,
    #то вместо количество месяцев запаса товара, заданным в начале - берем срок доставки и срок производства из карточки товара
    if table_zayavki['Страна регистрации'][j]!='РОССИЯ':
        max2=max1
    else:
        if max1<table_zayavki['Средний остаток на Инфор за последние полгода'][j]:
            max2=max1
        else:
            max2=table_zayavki['Среднее по сглаженному'][j]+std_otkl*normstobr*((table_zayavki['Срок производства (дней)'][j]+table_zayavki['Срок доставки (дней)'][j])/30)**(0.5)

    #полученное значение округляем до целого значения
    table_zayavki['макс2'][j]=int(max2) + (1 if max2%1 >= 0.5 else 0)  # тут меняю int на float


    #max3
    #обнуляем макс, если:
        #Если [Под заказ]=Да
        #Если [Стоп заказ] = Да и [Остаток на Инфоре на сегодня]=0
        #Если [Количество заказов за анализируемый период] <  [Количество месяцев с заказами]
        #Если за влияющий период сумма заявок по сглаженным данным = 0



    if table_zayavki['Под заказ'][j]=="Да" or (table_zayavki['Больше не поставляется'][j]=="Да" and table_zayavki['Остаток на Инфоре на сегодня'][j]==0):
        max3=0
    else:
        if table_zayavki['Количество заказов за анализируемый период'][j]<min_kolvo_zakazov:
            max3=0
        else:
            if table_zayavki['Сумма продаж за влияющий период с учетом всплесков'][j]==0:
                prom_max3=0
            else:
                prom_max3=max2

            #если полученный макс > [максимальный запас], то обрезаем до максимального запаса

            if prom_max3/table_zayavki['Среднее по сглаженному'][j]>max_zapas:
                max3=table_zayavki['Среднее по сглаженному'][j]*max_zapas
            else:
                max3=max2

    table_zayavki['макс3'][j]=int(max3) + (1 if max3%1 >= 0.5 else 0)


    #min1
    #рассчитываем количество, нужное на период доставки от инфора до склада, для которого рассчитываем

    num_1=table_zayavki['Среднее по сглаженному'][j]*obwee_vremya_na_dostavku/30
    #полученное значение округляем до ближайшего целого в большую сторону
    min1=int(int(num_1) + (1 if num_1%1 > 0 else 0)) # тут меняю int на float
    table_zayavki['мин1'][j]=min1

    #min2

    #обнуляем мин если количество заказов < минимального количество заказов за период
    if table_zayavki['Количество заказов за анализируемый период'][j]<min_kolvo_zakazov:
        min2=0
    else:
        if min1>=max3:
            if max3==0:
                min2=0
            else:
                min2=max3-1
        else:
            min2=min1

    table_zayavki['мин2'][j]=min2

    #min3
     #обнуляем макс, если:
        #Если [Под заказ]=Да
        #Если [Стоп заказ] = Да и [Остаток на Инфоре на сегодня]=0
        #Если [Количество заказов за анализируемый период] <  [Количество месяцев с заказами]


    if table_zayavki['Под заказ'][j]=="Да" or (table_zayavki['Больше не поставляется'][j]=="Да" and table_zayavki['Остаток на Инфоре на сегодня'][j]==0):
        min3=0
    else:
        min3=min2

    table_zayavki['мин3'][j]=min3
    #min4

    #Проверяем, чтобы мин был не меньше [Минимальный остаток не больше максимального остатка] макса

    if max3==0:
        min4=0
    else:
        if min3/max3<min_menshe_maxa_ne_bolee_chem_na:
            delo=max3*min_menshe_maxa_ne_bolee_chem_na
            min4=int(delo) + (1 if delo%1 >= 0.5 else 0)
        else:
            min4=min3

    table_zayavki['мин4'][j]=min4





#сегодняшняя дата, для указания в наименовании файла
month=datetime.datetime.now().month
year=datetime.datetime.now().year
day=datetime.datetime.now().day
data=str(day)+'_'+str(month)+'_'+str(year)


#данные для загрузки мин-максов в 1с

table_zayavki=table_zayavki[table_zayavki['макс3']!=0]
df1 = table_zayavki[['Артикул','мин4', 'макс3']]
df1.rename(columns={'мин4': 'Минимальный остаток'}, inplace=True)
df1.rename(columns={'макс3': 'Максимальный остаток'}, inplace=True)

df1.to_excel(name+'_'+data+'.xlsx')

<ipython-input-5-09789aae0b73>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_zayavki['Месяцев с создания'][i]=(dt_now-datetime.datetime.strptime(table_zayavki['Дата создания'][i], '%d.%m.%Y %H:%M:%S')).days/30
<ipython-input-5-09789aae0b73>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table_zayavki['Среднее количество шт в заказах за анализируемый период с учетом даты заведения'][i]=table_zayavki['Итого'][i]/an_period
<ipython-input-5-09789aae0b73>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.or

In [ ]:
result_df['Минимальный остаток'].fillna(0, inplace=True)
result_df['Максимальный остаток'].fillna(0, inplace=True)
result_df['Максимальный остаток'].replace(0, 1, inplace=True)